In [1]:
import numpy as np
import PIL
import matplotlib.pyplot as plt
import torch
import openpifpaf
import cv2
import os
import pandas as pd
import ultralytics
from ultralytics import YOLO

ultralytics.checks()

Ultralytics 8.3.145  Python-3.10.16 torch-1.13.1+cpu CPU (11th Gen Intel Core(TM) i7-1165G7 2.80GHz)
Setup complete  (8 CPUs, 63.7 GB RAM, 538.2/953.8 GB disk)


In [2]:
print('OpenPifPaf version', openpifpaf.__version__)
print('PyTorch version', torch.__version__)

OpenPifPaf version 0.13.11
PyTorch version 1.13.1+cpu


In [ ]:
yolo_model = YOLO('C:\Austin\Projects\MS_Thesis\Code\yolo11n.pt')
pose_predictor = openpifpaf.Predictor(checkpoint='shufflenetv2k16')

Downloading: "http://github.com/openpifpaf/torchhub/releases/download/v0.13/shufflenetv2k16-210820-232500-cocokp-slurm726069-edge513-o10s-7189450a.pkl" to C:\Users\Austin Paulraj/.cache\torch\hub\checkpoints\shufflenetv2k16-210820-232500-cocokp-slurm726069-edge513-o10s-7189450a.pkl
100%|██████████| 38.9M/38.9M [00:07<00:00, 5.13MB/s]
